In [39]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [40]:
movies=pd.read_csv('../lectures/data/movies.csv', encoding = "ISO-8859-1")
links=pd.read_csv('../lectures/data/links.csv', encoding = "ISO-8859-1")

In [41]:
pd.options.display.max_rows=12

In [42]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
...,...,...,...
27272,131252,Forklift Driver Klaus: The First Day on the Jo...,Comedy|Horror
27273,131254,Kein Bund fÃ¼r's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy


In [43]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
5,6,113277,949.0
...,...,...,...
27272,131252,289477,9677.0
27273,131254,466713,4436.0
27274,131256,277703,9274.0


In [44]:
merged = pd.merge(movies, links, on='movieId')

In [45]:
merged

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0
5,6,Heat (1995),Action|Crime|Thriller,113277,949.0
...,...,...,...,...,...
27272,131252,Forklift Driver Klaus: The First Day on the Jo...,Comedy|Horror,289477,9677.0
27273,131254,Kein Bund fÃ¼r's Leben (2007),Comedy,466713,4436.0
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,277703,9274.0


In [46]:
imdb_url = 'http://www.imdb.com/title/tt'
themoviedb_url = 'https://www.themoviedb.org/movie/'

In [47]:
def get_all_ratings(url):
    limit = 10
    ratings = []

    for index, row in merged.iterrows():
        if (index == limit):
            break
        req = requests.get(url + str(row.imdbId) + '/')
        if not (req.status_code == 200):
            return False
        
        soup = BeautifulSoup(req.text, 'html5lib')
        
        rating = soup.find(itemprop='ratingValue').get_text()
        ratings.append(rating)
        
    return ratings


In [ ]:
def get_ratings_from_themoviedb(url):
    limit = 10
    ratings = []
    
    for index, row in merged.iterrows():
        if(index == limit):
            break
        req = requests.get(url + str(row.tmdbId))
        
        if not (req.status_code == 200):
            break
        
        soup = BeautifulSoup(req.text, 'html5lib')
        
        rating = soup.findAll('div', {'class': 'user_score_chart'})[0]['data-percent']
        ratings.append(rating)
    
    return ratings;

In [ ]:
imdb_ratings = get_all_ratings(imdb_url)
tmdb_ratings = get_ratings_from_themoviedb(themoviedb_url)

In [ ]:
selected = merged[:10]
selected['imdb_ratings'] = imdb_ratings
selected['tmfb_ratings'] = tmdb_ratings
selected